mpdg_SOMOCLU.ipynb

Builds and runs the SOM.\
Adjust SOM parameters in the *set parameters* cell.

In [ ]:
import os
from glob import glob

from astropy.io import fits
from astropy.table import Table, hstack, vstack

import fitsio
from fitsio import FITS

import numpy as np
from numpy.lib.recfunctions import structured_to_unstructured, append_fields

import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

from matplotlib.patches import Rectangle

from cycler import cycler
plt.rcParams.update({
    'figure.figsize'   : (12, 10),
    'font.family'      :'serif',
    'font.size'        : 24,
    'font.serif'       :'Georgia',
    'axes.labelsize'   :'large',
    'mathtext.fontset' :'stix',
    'axes.linewidth'   : 5,
    'xtick.direction'  :'in',
    'ytick.direction'  :'in',
    'xtick.major.size' : 7,
    'ytick.major.size' : 7,
    'xtick.major.width': 3,
    'ytick.major.width': 3,
    'xtick.minor.size' : 5,
    'ytick.minor.size' : 5,
    'xtick.minor.width': 2,
    'ytick.minor.width': 2,
    'lines.linewidth'  : 3,
    'axes.prop_cycle'  : cycler('color', ['red', 'orange', 'teal', 'skyblue']),
})

def median_of_dist(x, p_x):
    dx = x[1] - x[0]

    cdf = np.array([np.sum(dx * p_x[:i]) for i in range(len(x))])
    cdf_half = cdf <= 1/2

    approx_median = x[cdf_half][-1]

    return approx_median

def median_1sigma_of_dist(x, p_x):
    dx = x[1] - x[0]

    cdf = np.array([np.sum(dx * p_x[:i]) for i in range(len(x))])
    cdf_l68 = cdf <= 0.16
    cdf_u68 = cdf <= 0.84

    approx_l68 = x[cdf_l68][-1]
    approx_u68 = x[cdf_u68][-1]

    return approx_l68, approx_u68

def variance_of_distribution(x, p_x):
    
    dx = x[1] - x[0]

    E_x2 = np.sum(x**2 * p_x * dx)
    E2_x = np.sum(x * p_x * dx)**2

    return E_x2 - E2_x

In [ ]:
folder_name = '/Users/leo/Projects/mass_profile_dg/saved_SOMs/15Aug24_test/'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

In [ ]:
# #Load in KiDS data
def load_KiDS_data(path):

    # cat_section = fitsio.read(path, ext = 1)
    cat_section = FITS(path)[1][:]
    
    mag_cut      = cat_section['r_mag'] < 20.5
    redshift_cut = cat_section['redshift'] < 0.4

    cat_section_cut = cat_section[redshift_cut & mag_cut]

    colnames = cat_section.dtype.names
    columns = ['r_mag']
    columns +=[key for key in colnames\
               if key.endswith('col')]
    columns +=['half_light_radius']
    # columns +=['redshift']

    cat_section_selected = cat_section_cut[columns]
    
    coordinate_columns = ['ID', 'KiDS_tile', 'ra', 'dec']
    
    cat_section_coordinates = cat_section_cut[coordinate_columns]

    return cat_section_selected, cat_section_coordinates

In [ ]:
#Load in KiDS data
training_data_secs_path = '/Users/leo/Projects/mass_profile_dg/data_products/SOM_catalogs/KiDS/KiDS_SOM_panchrom_07Aug24/KiDS_SOM_panchrom_07Aug24'
training_data_sec1,  coordinate_info_sec1  = load_KiDS_data(f'{training_data_secs_path}_sec1.fits')
training_data_sec2,  coordinate_info_sec2  = load_KiDS_data(f'{training_data_secs_path}_sec2.fits')
training_data_sec3,  coordinate_info_sec3  = load_KiDS_data(f'{training_data_secs_path}_sec3.fits')
training_data_sec4,  coordinate_info_sec4  = load_KiDS_data(f'{training_data_secs_path}_sec4.fits')
training_data_sec5,  coordinate_info_sec5  = load_KiDS_data(f'{training_data_secs_path}_sec5.fits')
training_data_sec6,  coordinate_info_sec6  = load_KiDS_data(f'{training_data_secs_path}_sec6.fits')
training_data_sec7,  coordinate_info_sec7  = load_KiDS_data(f'{training_data_secs_path}_sec7.fits')
training_data_sec8,  coordinate_info_sec8  = load_KiDS_data(f'{training_data_secs_path}_sec8.fits')
training_data_sec9,  coordinate_info_sec9  = load_KiDS_data(f'{training_data_secs_path}_sec9.fits')
training_data_sec10, coordinate_info_sec10 = load_KiDS_data(f'{training_data_secs_path}_sec10.fits')
training_data_sec11, coordinate_info_sec11 = load_KiDS_data(f'{training_data_secs_path}_sec11.fits')

In [ ]:
original_training_data = np.concatenate([training_data_sec1, 
                                        training_data_sec2, 
                                        training_data_sec3, 
                                        training_data_sec4, 
                                        training_data_sec5, 
                                        training_data_sec6, 
                                        training_data_sec7, 
                                        training_data_sec8, 
                                        training_data_sec9, 
                                        training_data_sec10,
                                        training_data_sec11], axis = 0)

coordinate_info = np.concatenate([coordinate_info_sec1, 
                                  coordinate_info_sec2, 
                                  coordinate_info_sec3, 
                                  coordinate_info_sec4, 
                                  coordinate_info_sec5, 
                                  coordinate_info_sec6, 
                                  coordinate_info_sec7, 
                                  coordinate_info_sec8, 
                                  coordinate_info_sec9, 
                                  coordinate_info_sec10,
                                  coordinate_info_sec11], axis = 0)

training_data_names = original_training_data.dtype.names
original_training_data_array = original_training_data.copy()
original_training_data_array = structured_to_unstructured(original_training_data_array)
training_data = original_training_data.copy()
training_data = structured_to_unstructured(training_data)

In [ ]:
GAMA_cat = FITS('/Users/leo/Projects/mass_profile_dg/data_products/SOM_catalogs/GAMA/GAMA_SOM_KiDSphotom_08Aug24.fits')[1][:]

In [ ]:
#Select the needed labeling data

GAMA_redshift_cut = GAMA_cat['spec_z'] < 0.4

labels_cat = GAMA_cat[GAMA_redshift_cut]

labeling_data_columns = ['r_mag']
labeling_data_columns +=[key for key in labels_cat.dtype.names if key.endswith('col')]
labeling_data_columns +=['half_light_radius']
# labeling_data_columns +=['BPZ']

labeling_data_inputs = labels_cat[labeling_data_columns]

labeling_data_inputs = structured_to_unstructured(labeling_data_inputs)

labeling_data_outputs = labels_cat[['spec_z', 'mstar']]

labeling_data_outputs = structured_to_unstructured(labeling_data_outputs)

labeling_parameter_names = ['spec_z','mstar']


In [ ]:
print(f'Len of training data:\t\t{len(training_data)}')
print(f'Dimension =||=:\t\t\t{np.shape(training_data)[-1]}')
print(f'Len of labeling data inputs:\t{len(labeling_data_inputs)}')
print(f'Dimension =||=:\t\t\t{np.shape(labeling_data_inputs)[-1]}')
print(f'Len of labeling data outputs:\t{len(labeling_data_outputs)}')
print(f'Dimension =||=:\t\t\t{np.shape(labeling_data_outputs)[-1]}')

In [ ]:
fig = plt.figure(figsize = (30, 20), constrained_layout = True)

for i in range(5):
    for j in range(8):

        ax = fig.add_subplot(5, 8, i*8 + j + 1)

        try:
            # bins = ax.hist(training_data[training_data_names[i*8 + j]], density = True, bins = 'auto', color = ('red', 0.5))[1]
            bins = ax.hist(training_data[:, i*8 + j], density = True, bins = 'auto', color = ('red', 0.5))[1]
            # bins = 'auto'
            ax.hist(labeling_data_inputs[:, i*8 + j], density = True, bins = bins, histtype = 'step', color = 'black', linewidth = 3)
            # if i*8+j+1 != 37: ax.set_xlim(-1, 3)
            ax.set_xlabel(training_data_names[i*8 + j])
            ax.set_yticklabels([])
        except:
            # continue
            ax.axis('off')

GAMA_mappable = plt.plot([], [], color = 'black', linewidth = 3, label = 'GAMA')
KiDS_mappable = plt.hist([], color = ('red', 0.5), label = 'KiDS')

fig.legend(loc = 'lower right', frameon = False, fontsize = 32)
fig.savefig(f'{folder_name}/data_dist.png', bbox_inches = 'tight')

In [ ]:
import somoclu

In [ ]:
#set parameters
n_columns      = 48
n_rows         = 48

maptype        = 'toroid'
gridtype       = 'hexagonal'

initialization = 'pca'

neighborhood   = 'gaussian'
std_coeff      = 0.5

verbose        = 2

compactsupport = False

#training parameters
epochs        = 50

radius0       = 0
radiusN       = 1
radiuscooling = 'exponential'

scale0        = 0.25
scaleN        = 0.01
scalecooling  = 'exponential'

In [ ]:
SOM = somoclu.Somoclu(n_columns      = n_columns,
                      n_rows         = n_rows,
                      maptype        = maptype,
                      gridtype       = gridtype,
                      initialization = initialization,
                      neighborhood   = neighborhood,
                      std_coeff      = std_coeff,
                      verbose        = verbose,
                      compactsupport = compactsupport)

In [ ]:
#Select subset of data
data_cut = 1000000 #set to 'all' to use all data, otherwise set to an integer

print(f'Initial training data len:\t{len(original_training_data)}')
if data_cut != 'all':
    randomized_idx = np.arange(0, len(original_training_data))
    np.random.shuffle(randomized_idx)
    randomized_data_idx = randomized_idx[:data_cut]
    training_data = original_training_data_array[randomized_data_idx]
print(f'Subset training data len:\t{len(training_data)}')

In [ ]:
SOM.train(data = training_data,
          epochs = epochs,
          radius0 = radius0, radiusN = radiusN, radiuscooling = radiuscooling,
          scale0 = scale0,   scaleN = scaleN,   scalecooling = scalecooling)

In [ ]:
SOM.codebook = np.load(f'{folder_name}SOM_codebook.npy', allow_pickle = True)

In [ ]:
np.save(f'{folder_name}/SOM_codebook', SOM.codebook, allow_pickle = True)

In [ ]:
#Visualize trained SOM
hexmesh = np.meshgrid(np.linspace(0, n_rows - 1, n_rows),
                       np.linspace(0, n_columns - 1, n_columns))

hexmesh[0][1::2] += 1/2

fig = plt.figure(figsize = (80, 60), constrained_layout = True)
transposed_codebook = np.transpose(SOM.codebook, [1, 0, 2])

for i in range(5):
    for j in range(8):

        ax = fig.add_subplot(5, 8, i*8 + j + 1)

        try:
            map = ax.scatter(*hexmesh, c = transposed_codebook[..., i*8 + j], marker = 'h', cmap = 'jet', s = 320)
            fig.colorbar(mappable = map, ax = ax, location = 'bottom', label = training_data_names[i*8 + j], shrink = 0.9, pad = 0)
        except: pass
        ax.axis('off')

fig.savefig(f'{folder_name}/trained_SOM.png')

In [ ]:
training_activation_map = SOM.get_surface_state(data = original_training_data_array[0:1000])
training_bmus = SOM.get_bmus(activation_map = training_activation_map)

for step in range(1, int(len(original_training_data_array)/1000) + 1):
    i_lo = step * 1000
    i_hi = (step + 1) * 1000

    training_activation_map = SOM.get_surface_state(data = original_training_data_array[i_lo:i_hi])
    set_bmus = SOM.get_bmus(activation_map = training_activation_map)
    training_bmus = np.append(training_bmus, set_bmus, axis = 0)

In [ ]:
labeling_activation_map = SOM.get_surface_state(data = labeling_data_inputs)
labeling_bmus           = SOM.get_bmus(activation_map = labeling_activation_map)

In [ ]:
steps = 250
min_z = 0; max_z = 0.4
min_mstar = 6; max_mstar = 12

def gauss(x, mu, sigma): return 1/np.sqrt(2 * np.pi * sigma**2) * np.exp(-(x - mu)**2/(2 * sigma**2))

redshift_sigma = 0.025
redshift_gaussian = 1/np.sqrt(2 * np.pi * redshift_sigma ** 2) *\
                    np.exp(-(np.linspace(min_z, max_z, steps) - (max_z - min_z)/2) ** 2/(2 * redshift_sigma**2))
redshift_gaussian /= np.sum(redshift_gaussian)

mstar_sigma = 0.189
mstar_gaussian = 1/np.sqrt(2 * np.pi * mstar_sigma ** 2) *\
                 np.exp(-(np.linspace(min_mstar, max_mstar, steps) - (max_mstar + min_mstar)/2) ** 2/(2 * mstar_sigma**2))
mstar_gaussian /= np.sum(mstar_gaussian)

In [ ]:
n_params = np.shape(labeling_data_outputs)[1]

labeled_map_values = np.empty((n_rows, n_columns, n_params), dtype = object)
for index, _ in np.ndenumerate(labeled_map_values):
    labeled_map_values[index] = []

for vector_index in range(len(labeling_bmus)):
    for param in range(n_params):
        labeled_map_values[*labeling_bmus[vector_index], param].append(labeling_data_outputs[vector_index, param])

labeled_map_z = np.empty((n_rows, n_columns, steps))
bins = np.linspace(min_z, max_z, steps)
for cell_idx, cell in np.ndenumerate(labeled_map_values[..., 0]):
    values = np.histogram(np.array(cell), bins = bins)[0]
    dist_in_cell = np.convolve(values, redshift_gaussian, mode = 'same')
    labeled_map_z[cell_idx] = dist_in_cell/np.sum(dist_in_cell * (bins[1] - bins[0]))

labeled_map_mstar = np.empty((n_rows, n_columns, steps))
bins = np.linspace(min_mstar, max_mstar, steps)
for cell_idx, cell in np.ndenumerate(labeled_map_values[..., 1]):
    values = np.histogram(np.log10(cell), bins = bins)[0]
    dist_in_cell = np.convolve(values, mstar_gaussian, mode = 'same')
    labeled_map_mstar[cell_idx] = dist_in_cell/np.sum(dist_in_cell * (bins[1] - bins[0]))


In [ ]:
x = np.linspace(min_z, max_z, steps)
p_x = labeled_map_z[np.random.randint(0, 48), np.random.randint(0, 48)]

plt.plot(x, p_x)
plt.axvline(median_of_dist(x, p_x))
plt.axvline(median_1sigma_of_dist(x, p_x)[0])
plt.axvline(median_1sigma_of_dist(x, p_x)[1])

In [ ]:
labeled_map     = np.zeros((n_rows, n_columns, n_params))
labeled_map_l68 = np.zeros((n_rows, n_columns, n_params))
labeled_map_u68 = np.zeros((n_rows, n_columns, n_params))

x = np.linspace(min_z, max_z, steps)
for index, _ in np.ndenumerate(labeled_map_z[..., 0]):
    p_x = labeled_map_z[*index]/np.sum(labeled_map_z[*index] * (x[1] - x[0]))
    median = median_of_dist(x, p_x)
    l68, u68 = median_1sigma_of_dist(x, p_x)
    labeled_map[*index, 0]     = median       if median > min_z else np.nan
    labeled_map_l68[*index, 0] = median - l68 if median > min_z else np.nan
    labeled_map_u68[*index, 0] = u68 - median if median > min_z else np.nan

x = np.linspace(min_mstar, max_mstar, steps)
for index, _ in np.ndenumerate(labeled_map_mstar[..., 0]):
    p_x = labeled_map_mstar[*index]/np.sum(labeled_map_mstar[*index] * (x[1] - x[0]))
    median = median_of_dist(x, p_x)
    l68, u68 = median_1sigma_of_dist(x, p_x)
    labeled_map[*index, 1]     = median       if median > min_mstar else np.nan
    labeled_map_l68[*index, 1] = median - l68 if median > min_mstar else np.nan
    labeled_map_u68[*index, 1] = u68 - median if median > min_mstar else np.nan

In [ ]:
folder_name

In [ ]:
np.save(f'{folder_name}/labeled_map_from_dist', labeled_map, allow_pickle = True)
np.save(f'{folder_name}/labeled_map_l68', labeled_map_l68, allow_pickle = True)
np.save(f'{folder_name}/labeled_map_u68', labeled_map_u68, allow_pickle = True)

In [ ]:
n_params = np.shape(labeling_data_outputs)[1]

labeled_map_values = np.empty((n_rows, n_columns, n_params), dtype = object)
for index, _ in np.ndenumerate(labeled_map_values):
    labeled_map_values[index] = []

for vector_index in range(len(labeling_bmus)):
    for param in range(n_params):
        labeled_map_values[*labeling_bmus[vector_index], param].append(labeling_data_outputs[vector_index, param])

labeled_map = np.empty((n_rows, n_columns, n_params), dtype = float)
for index, values in np.ndenumerate(labeled_map_values):
    if (len(values) > 0):
        labeled_map[index] = np.median(labeled_map_values[index])
    else: labeled_map[index] = np.nan

In [ ]:
## make mstar log
# labeled_map[..., 1] = np.log10(labeled_map[..., 1])

In [ ]:
#Visualize labeled SOM
hexmesh = np.meshgrid(np.linspace(0, n_rows, n_rows),
                      np.linspace(0, n_columns, n_columns))

hexmesh[0][1::2] += 1/2

fig = plt.figure(figsize = (25, 6.5), constrained_layout = True)

for i in range(n_params):
    ax = fig.add_subplot(1, 5, i + 1)

    if i == 0:
        vmin = 0; vmax = 0.4
    if i == 1:
        vmin = 7; vmax = 11
    map = ax.scatter(*hexmesh, c = labeled_map[..., i], marker = 'h', cmap = 'jet', s = 60,
                     vmin = vmin, vmax = vmax)
    ax.axis('off')
    fig.colorbar(mappable = map, ax = ax, location = 'bottom', label = ['redshift', 'log_mstar'][i], shrink = 0.9)

# fig.savefig(f'{folder_name}/labeled_SOM.png', bbox_inches = 'tight')

In [ ]:
occupancy_map = np.empty((n_rows, n_columns), dtype = float)
for index, values in np.ndenumerate(labeled_map_values[..., 0]):
    if len(values) > 0:
        occupancy_map[index] = len(values)
    else: occupancy_map[index] = np.nan

fig = plt.figure(figsize = (7.5, 9), constrained_layout = True)
ax = fig.add_subplot()
ax.axis('off')

map = ax.scatter(*hexmesh, c = np.log10(occupancy_map), marker = 'h', cmap = 'jet', s = 180)
fig.colorbar(mappable = map, ax = ax, location = 'bottom', label = f'log10(galaxies/cell)\nMean occupancy:{np.nanmean(occupancy_map):.1f}', shrink = 0.9)

fig.savefig(f'{folder_name}/occupancy_map.png', bbox_inches = 'tight')

In [ ]:
predicted_values_training = np.empty((len(original_training_data_array), n_params))
predicted_values_labeling = np.empty((len(labeling_data_inputs), n_params))

predicted_l68_training = np.empty((len(original_training_data_array), n_params))
predicted_l68_labeling = np.empty((len(labeling_data_inputs), n_params))

predicted_u68_training = np.empty((len(original_training_data_array), n_params))
predicted_u68_labeling = np.empty((len(labeling_data_inputs), n_params))

for i, bmu in enumerate(training_bmus):
    predicted_values_training[i] = labeled_map[*bmu]
    predicted_l68_training[i] = labeled_map_l68[*bmu]
    predicted_u68_training[i] = labeled_map_u68[*bmu]

for i, bmu in enumerate(labeling_bmus):
    predicted_values_labeling[i] = labeled_map[*bmu]
    predicted_l68_labeling[i] = labeled_map_l68[*bmu]
    predicted_u68_labeling[i] = labeled_map_u68[*bmu]

In [ ]:
fig = plt.figure(figsize = (24, 10))

ax_l = fig.add_subplot(1, 2, 1)

bins = ax_l.hist(labeling_data_outputs[:, 0], bins = 50, density = True,
                 color = 'green', alpha = 0.25, label = 'GAMA true')[1]

ax_l.hist(predicted_values_labeling[:, 0], bins = bins, density = True,
          histtype = 'step', color = 'black', label = 'GAMA Predicted')

ax_l.set_xlabel('Redshift')
ax_l.set_ylabel('$dN/dz$')
ax_l.legend(frameon = False)

ax_r = fig.add_subplot(1, 2, 2)

bins = ax_r.hist(np.log10(labeling_data_outputs[:, 1]), bins = 50, density = True,
                 color = 'green', alpha = 0.25, label = 'GAMA true')[1]

ax_r.hist(predicted_values_labeling[:, 1], bins = bins, density = True,
          histtype = 'step', color = 'black', label = 'GAMA Predicted')
ax_r.set_xlabel('$\log_{10}(M_*/M_{\odot})$')
ax_r.set_ylabel('$dN/dM_*$')
ax_r.legend(frameon = False)

fig.savefig(f'{folder_name}/distribution.png', bbox_inches = 'tight')

In [ ]:
fig = plt.figure(figsize = (24, 16))

ax_l = fig.add_subplot(1, 2, 1)

hb_l = ax_l.hexbin(labeling_data_outputs[:, 0],
                   predicted_values_labeling[:, 0],
                   mincnt = 1, cmap = 'jet',
                   vmin = 1, vmax = 150)

ax_l.axline([0.2, 0.2], slope = 1, color = 'red')
ax_l.set_xlim(-0.01, 0.405)
ax_l.set_ylim(-0.01, 0.405)
ax_l.set_xticks(np.arange(0, 0.45, 0.05))

ax_l.set_xlabel('GAMA $z$\nTrue')
ax_l.set_ylabel('Predicted\nSOM $z$')

z_MAD = np.mean(np.abs(labeling_data_outputs[:, 0] - predicted_values_labeling[:, 0]))
z_RMSE = np.sqrt(np.mean(labeling_data_outputs[:, 0]**2 - predicted_values_labeling[:, 0] ** 2))

ax_l.text(0.65, 0.05, f'MAD: {z_MAD:.3f}\nRMSE: {z_RMSE:.3f}',
          transform = ax_l.transAxes)


ax_r = fig.add_subplot(1, 2, 2)


hb_r = ax_r.hexbin(np.log10(labeling_data_outputs[:, 1]),
                   predicted_values_labeling[:, 1],
                   mincnt = 1, cmap = 'jet',
                   vmin = 1, vmax = 150)

ax_r.axline([10, 10], slope = 1, color = 'red')
ax_r.set_xlim(7, 11.2)
ax_r.set_ylim(7, 11.2)
ax_r.set_xticks(np.arange(7, 11.5, 0.5))

ax_r.set_xlabel('GAMA $\log_{10} (M_*/M_{\odot})$\nTrue')
ax_r.set_ylabel('Predicted\nSOM $\log_{10} (M_*/M_{\odot})$')

Mstar_MAD = np.nanmean(np.abs(np.log10(labeling_data_outputs[:, 1]) - predicted_values_labeling[:, 1]))
Mstar_RMSE = np.sqrt(np.mean(np.log10(labeling_data_outputs[:, 1]) - predicted_values_labeling[:, 1]) ** 2)

ax_r.text(0.65, 0.05, f'MAD: {Mstar_MAD:.3f} dex\nRMSE: {Mstar_RMSE:.3f} dex',
          transform = ax_r.transAxes)

fig.subplots_adjust(wspace = 0.25)

fig.colorbar(mappable = hb_l, location = 'bottom', ax = [ax_l, ax_r], shrink = 0.5)

fig.savefig(f'{folder_name}/comparison.png', bbox_inches = 'tight')

In [ ]:
# np.save(f'/data2/lsajkov/mpdg/saved_soms/30Jul24_mag_cut/r_19p5/codebook', SOM.codebook, allow_pickle = True)
np.save(f'{folder_name}/labeled_map_values', labeled_map_values, allow_pickle = True)
np.save(f'{folder_name}/labeled_map', labeled_map, allow_pickle = True)
np.save(f'{folder_name}/data_bmus', training_bmus, allow_pickle = True)#SOM.bmus, allow_pickle = True)
np.save(f'{folder_name}/labeling_bmus', labeling_bmus, allow_pickle = True)

In [ ]:
predicted_values_cat = FITS(f'{folder_name}/predicted_catalog.fits', 'rw')
predicted_values_cat.write(append_fields(coordinate_info, ['redshift_pred', 'redshift_pred_l68', 'redshift_pred_u68',
                                                           'mstar_pred', 'mstar_pred_l68', 'mstar_pred_u68'],
                                                           [predicted_values_training[:, 0], predicted_l68_training[:, 0], predicted_u68_training[:, 0],
                                                            predicted_values_training[:, 1], predicted_l68_training[:, 1], predicted_u68_training[:, 1]]))
predicted_values_cat.close()

# labeled_training_cat = Table(original_training_data.copy())
# labeled_training_cat.add_columns([predicted_values_training[:, 0], predicted_values_training[:, 1]],
#                                  names = ['redshift_pred', 'mstar_pred'])

# labeled_training_cat.write(f'{folder_name}/predicted_catalog.fits',
#                            overwrite = False)